In [1]:
!pip install -qU langchain-openai
!pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00


Additionally, the install the newspaper3k package

In [7]:
!pip install -q newspaper3k python-dotenv

In [8]:
!pip install -q lxml_html_clean

In your Python script or notebook, set the API key as an environment variable with `OPENAI_API_KEY` name. In order to set it from a `.env` file, you can use the `load_dotenv` function.

In [9]:
import os
import json
# for local usage
#from dotenv import load_dotenv
#load_dotenv()

# for colab
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Let us use the URL of a news article to generate a summary. The following code fetches articles from a list of URLs using the `requests` library with a custom User-Agent header. It then extracts the title and text of each article using the `newspaper` library.

In [12]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://apnews.com/article/jimmy-carter-dies-18c198c20352c835bca3eec276020dd7"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)

    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()

        print(f"Title: {article.title}")
        print(f"Text: {article.text}")

    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Jimmy Carter, 39th US president, has died at 100
Text: ▶ Follow live updates as the world reacts to the death of Former President Jimmy Carter

ATLANTA (AP) — Jimmy Carter, the peanut farmer who won the presidency in the wake of the Watergate scandal and Vietnam War, endured humbling defeat after one tumultuous term and then redefined life after the White House as a global humanitarian, has died. He was 100 years old.

The longest-lived American president died on Sunday, roughly 22 months after entering hospice care, at his home in the small town of Plains, Georgia, where he and his wife, Rosalynn, who died at 96 in November 2023, spent most of their lives, The Carter Center said.

“Our founder, former U.S. President Jimmy Carter, passed away this afternoon in Plains, Georgia,” the center said on the social media platform X. It added in a statement that he died peacefully, surrounded by his family.

As reaction poured in from around the world, President Joe Biden mourned Carter’

The next code imports essential classes and functions from the LangChain and sets up a `ChatOpenAI` instance with a temperature of 0 for controlled response generation. Additionally, it imports chat-related message schema classes, which enable the smooth handling of chat-based tasks. The following code will start by setting the prompt and filling it with the article’s content.

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

Write a summary of the previous article.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

The `HumanMessage` is a structured data format representing user messages within the chat-based interaction framework. The ChatOpenAI class is utilized to interact with the AI model, while the HumanMessage schema provides a standardized representation of user messages. The template consists of placeholders for the article's title and content, which will be substituted with the actual `article_title` and `article_text`. This process simplifies and streamlines the creation of dynamic prompts by allowing you to define a template with placeholders and then replace them with actual data when needed.

In [14]:
from langchain_openai import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

As we loaded the model and set the temperature to 0. We’d use the `chat()` instance to generate a summary by passing a single `HumanMessage` object containing the formatted prompt. The AI model processes this prompt and returns a concise summary:

In [16]:
summary = chat.invoke(messages)
print(summary.content)

Former President Jimmy Carter, the 39th president of the United States, passed away at the age of 100 in his hometown of Plains, Georgia, after spending 22 months in hospice care. Known for his humble beginnings as a peanut farmer and his presidency during a tumultuous period marked by the Watergate scandal and the Vietnam War, Carter's legacy extends far beyond his single term in office. He is remembered for his humanitarian efforts, including his work with The Carter Center, which he founded in 1982 to promote peace, democracy, and public health globally.

Carter's presidency was characterized by significant achievements, such as the Camp David Accords, which brokered peace between Egypt and Israel, and initiatives aimed at human rights and environmental protection. However, his administration faced challenges, including economic turmoil, high inflation, and the Iran hostage crisis, which ultimately led to his defeat in the 1980 election.

Post-presidency, Carter became a prominent g

If we want a bulleted list, we can modify a prompt and get the result.

In [17]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarizes online articles into bulleted lists.

Here's the article you need to summarize.

==================
Title: {article_title}

{article_text}
==================

Now, provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

# generate summary
summary = chat.invoke([HumanMessage(content=prompt)])
print(summary.content)

- **Jimmy Carter's Death**: Former President Jimmy Carter passed away at the age of 100 on Sunday in Plains, Georgia, after 22 months in hospice care.
- **Legacy**: Carter, the 39th U.S. president, was known for his humanitarian work post-presidency, winning the Nobel Peace Prize and establishing The Carter Center.
- **Family and Reactions**: He died peacefully surrounded by family. President Joe Biden mourned his loss, highlighting Carter's contributions to global health, peace, and human rights.
- **State Funeral**: Biden announced a state funeral for Carter on January 9, 2024, and declared it a National Day of Mourning, with flags at half-staff for 30 days.
- **Presidency Overview**: Carter's presidency (1977-1981) was marked by significant challenges, including the Iran hostage crisis and economic issues, leading to his defeat by Ronald Reagan.
- **Key Achievements**: Notable accomplishments included the Camp David Accords, deregulation of industries, and a focus on human rights in

If you want to get the summary in French, you can instruct the model to generate the summary in French language. However, please note that GPT-4's main training language is English and while it has a multilingual capability, the quality may vary for languages other than English. Here's how you can modify the prompt.

In [18]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarizes online articles into bulleted lists in French.

Here's the article you need to summarize.

==================
Title: {article_title}

{article_text}
==================

Now, provide a summarized version of the article in a bulleted list format, in French.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

# generate summary
summary = chat.invoke([HumanMessage(content=prompt)])
print(summary.content)

- **Décès de Jimmy Carter** : L'ancien président américain Jimmy Carter est décédé à l'âge de 100 ans, après avoir passé 22 mois en soins palliatifs à son domicile à Plains, en Géorgie.
- **Réactions mondiales** : Le président Joe Biden a exprimé sa tristesse, qualifiant Carter d'« extraordinaire leader, homme d'État et humanitaire ».
- **Héritage humanitaire** : Après sa présidence, Carter a redéfini sa vie en tant qu'humanitaire mondial, travaillant pour l'éradication des maladies, la paix, et les droits civils.
- **Carrière politique** : Carter, un démocrate modéré, a été élu président en 1976, promettant de ne pas tromper le peuple américain après le scandale du Watergate.
- **Réalisations en tant que président** : Il est connu pour avoir négocié un accord de paix au Moyen-Orient en 1978 entre l'Égypte et Israël, mais a également fait face à des défis économiques et à la crise des otages en Iran.
- **Échec électoral** : Sa présidence a été marquée par une inflation élevée et une cr